In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate,tune
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.Encoders.FIM import FIM_Encoder
from models.SFI import SFI_gating, SFI_gating_MultiView
from configs.ManualConfig import hparams

In [2]:
name='sfi'
hparams['k'] = 30
hparams['his_size'] = 50
hparams['select'] = 'gating'
hparams['onehot'] = False
hparams['device'] = 'cuda:1'
# hparams['threshold'] = 0.5

In [3]:
# hparams['validate'] = True
vocab, loaders = prepare(hparams, pin_memory=False)

[2021-05-03 17:49:38,712] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'vert_num': 18, 'subvert_num': 293, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:1', 'attrs': ['title'], 'k': 30, 'select': 'gating', 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'onehot': False, 'val_freq': 1, 'schedule': None}
[2021-05-03 17:49:38,713] INFO (root) preparing dataset...
[2021-05-03 17:49:42,809] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [4]:
encoder = FIM_Encoder(hparams, vocab)
# interactor = KNRM_Interactor()
hparams['name'] = '-'.join([name,encoder.name,hparams['select']])

# hparams['k'] = hparams['his_size']
# hparams['threshold'] = 0.3

# sfi = SFI_gating_MultiView(hparams, encoder, interactor).to(hparams['device'])
sfi = SFI_gating(hparams, encoder).to(hparams['device'])

In [15]:
sfi(record)

tensor([[[[7.4147e-03, 3.0695e-03, 4.5326e-03,  ..., 7.1669e-03,
           1.1234e-02, 6.6532e-02],
          [7.4147e-03, 3.0695e-03, 4.5326e-03,  ..., 7.1669e-03,
           1.1234e-02, 6.6532e-02],
          [7.4147e-03, 3.0695e-03, 4.5326e-03,  ..., 7.1669e-03,
           1.1234e-02, 6.6532e-02],
          ...,
          [7.4147e-03, 3.0695e-03, 4.5326e-03,  ..., 7.1669e-03,
           1.1234e-02, 6.6532e-02],
          [7.4147e-03, 3.0695e-03, 4.5326e-03,  ..., 7.1669e-03,
           1.1234e-02, 6.6532e-02],
          [7.4147e-03, 3.0695e-03, 4.5326e-03,  ..., 7.1669e-03,
           1.1234e-02, 6.6532e-02]],

         [[1.1226e-02, 4.6048e-03, 5.4867e-03,  ..., 1.2767e-02,
           1.4180e-02, 5.6625e-02],
          [1.1226e-02, 4.6048e-03, 5.4867e-03,  ..., 1.2767e-02,
           1.4180e-02, 5.6625e-02],
          [1.1226e-02, 4.6048e-03, 5.4867e-03,  ..., 1.2767e-02,
           1.4180e-02, 5.6625e-02],
          ...,
          [1.1226e-02, 4.6048e-03, 5.4867e-03,  ..., 1.2767

tensor([[-1.8876, -1.6809, -1.7411, -1.2787, -1.5671],
        [-1.6810, -1.6260, -1.1887, -2.2559, -1.5716],
        [-1.2298, -1.0235, -1.2136, -3.6655, -3.6655],
        [-2.5261, -1.5874, -1.3893, -1.4377, -1.4747],
        [-2.4394, -1.4356, -1.3503, -1.4243, -1.7432],
        [-1.8797, -1.5312, -1.5581, -1.9736, -1.2672],
        [-1.6400, -1.4804, -1.6827, -1.4567, -1.8351],
        [-1.6981, -1.9480, -1.1749, -2.2935, -1.3293],
        [-1.6659, -1.5857, -1.3000, -2.7382, -1.3132],
        [-1.4492, -1.7845, -1.4669, -2.0215, -1.4513]], device='cuda:1',
       grad_fn=<LogSoftmaxBackward>)

In [7]:
a = sfi.selectionProject.weight.clone()

In [8]:
hparams['epochs'] = 1
train(sfi, hparams, loaders)

[2021-05-03 17:50:46,937] INFO (root) training...
epoch 1 , step 290 , loss: 1.4668: 100%|██████████| 295/295 [00:25<00:00, 11.64it/s]
[2021-05-03 17:51:13,207] INFO (root) saved model of step 0, epoch 1 at data/model_params/sfi-fim-gating/demo_epoch1_step0_[hs=50,topk=30,attrs=title].model


SFI_gating(
  (encoder): FIM_Encoder(
    (embedding): Embedding(54316, 300, sparse=True)
    (ReLU): ReLU()
    (LayerNorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
    (DropOut): Dropout(p=0.2, inplace=False)
    (CNN_d1): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(1,))
    (CNN_d2): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (CNN_d3): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  )
  (softmax): Softmax(dim=-1)
  (interactor): FIM_Interactor(
    (SeqCNN3D): Sequential(
      (0): Conv3d(3, 32, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
      (1): ReLU()
      (2): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
      (3): Conv3d(32, 16, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
      (4): ReLU()
      (5): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
    )
  )
  (

In [9]:
sfi.selectionProject.weight == a

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:1')